In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
import csv
import functools
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
import string
import statistics

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
#from sklearn import svm
#from sklearn.svm import SVC
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [2]:
udf = pd.read_pickle("./4_COMB_FILE_UNDERSAMPLING_S1.pkl")
print(str(udf.shape))
odf = pd.read_pickle("./4_COMB_FILE_OVERSAMPLING_S1.pkl")
print(str(odf.shape))
df = pd.read_pickle("./4_COMB_FILE_UNBALANCED_S1.pkl")
print(str(df.shape))

(55160, 41)
(97866, 41)
(76513, 41)


In [3]:
# create bag of words for df
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(df.CLEAN_FOI_TEXT)
df_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
df_dtm.index = df.index
print(df_dtm.shape)

# train-test split for df
X_df = df_dtm
y_df = df["RECALL_CLASS"]

X_df_train, X_df_test, y_df_train, y_df_test = train_test_split(X_df, y_df, test_size = 0.33, random_state = 0)
print(X_df_train.shape)
print(X_df_test.shape)
print(y_df_train.shape)
print(y_df_test.shape)

(76513, 8789)
(51263, 8789)
(25250, 8789)
(51263,)
(25250,)


In [4]:
# create bag of words for udf
cv_u = CountVectorizer(stop_words='english')
data_cv_u = cv_u.fit_transform(udf.CLEAN_FOI_TEXT)
udf_dtm = pd.DataFrame(data_cv_u.toarray(), columns=cv_u.get_feature_names())
udf_dtm.index = udf.index
print(udf_dtm.shape)

# train-test split for udf
X_u = udf_dtm
y_u = udf["RECALL_CLASS"]

X_u_train, X_u_test, y_u_train, y_u_test = train_test_split(X_u, y_u, test_size = 0.33, random_state = 0)
print(X_u_train.shape)
print(X_u_test.shape)
print(y_u_train.shape)
print(y_u_test.shape)

(55160, 7897)
(36957, 7897)
(18203, 7897)
(36957,)
(18203,)


In [5]:
# create bag of words for odf
cv_o = CountVectorizer(stop_words='english')
data_cv_o = cv_o.fit_transform(odf.CLEAN_FOI_TEXT)
odf_dtm = pd.DataFrame(data_cv_o.toarray(), columns=cv_o.get_feature_names())
odf_dtm.index = odf.index
print(odf_dtm.shape)

# train-test split for odf
X_o = odf_dtm
y_o = odf["RECALL_CLASS"]

X_o_train, X_o_test, y_o_train, y_o_test = train_test_split(X_o, y_o, test_size = 0.33, random_state = 0)
print(X_o_train.shape)
print(X_o_test.shape)
print(y_o_train.shape)
print(y_o_test.shape)

(97866, 8762)
(65570, 8762)
(32296, 8762)
(65570,)
(32296,)


In [23]:
udf2 = pd.read_pickle("./4_COMB_FILE_UNDERSAMPLING_S2.pkl")
print(str(udf2.shape))
odf2 = pd.read_pickle("./4_COMB_FILE_OVERSAMPLING_S2.pkl")
print(str(odf2.shape))
df2 = pd.read_pickle("./4_COMB_FILE_UNBALANCED_S2.pkl")
print(str(df2.shape))

(4198, 41)
(4666, 41)
(4432, 41)


In [25]:
# create bag of words for odf2
cv_o2 = CountVectorizer(stop_words='english')
data_cv_o2 = cv_o.transform(odf2.CLEAN_FOI_TEXT)
odf_dtm2 = pd.DataFrame(data_cv_o2.toarray(), columns=cv_o.get_feature_names())
odf_dtm2.index = odf2.index
print(odf_dtm2.shape)

# train-test split for odf2
X_o2 = odf_dtm2
y_o2 = odf2["RECALL_CLASS"]

X_o2_train, X_o2_test, y_o2_train, y_o2_test = train_test_split(X_o2, y_o2, test_size = 0.33, random_state = 0)
print(X_o2_train.shape)
print(X_o2_test.shape)
print(y_o2_train.shape)
print(y_o2_test.shape)

(4666, 8762)
(3126, 8762)
(1540, 8762)
(3126,)
(1540,)


In [8]:
# running classifier models

In [6]:
# running naive-bayes classifiers
from sklearn.naive_bayes import GaussianNB

In [7]:
# for unbalanced class data
classifierdf = GaussianNB()
classifierdf.fit(X_df_train, y_df_train)
ydfPredNB = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredNB))
print('\n')
print(classification_report(y_df_test,ydfPredNB))

Unbalanced classes
[[3211  164]
 [3893 2094]]


              precision    recall  f1-score   support

           0       0.45      0.95      0.61      3375
           1       0.93      0.35      0.51      5987

    accuracy                           0.57      9362
   macro avg       0.69      0.65      0.56      9362
weighted avg       0.76      0.57      0.55      9362



In [8]:
# for undersampled class data
classifieru = GaussianNB()
classifieru.fit(X_u_train, y_u_train)
yuPredNB = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredNB))
print('\n')
print(classification_report(y_u_test,yuPredNB))

Undersampled classes
[[1806 1557]
 [ 169 3165]]


              precision    recall  f1-score   support

           0       0.91      0.54      0.68      3363
           1       0.67      0.95      0.79      3334

    accuracy                           0.74      6697
   macro avg       0.79      0.74      0.73      6697
weighted avg       0.79      0.74      0.73      6697



In [9]:
# for oversampled class data
classifier = GaussianNB()
classifier.fit(X_o_train, y_o_train)
yoPredNB = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredNB))
print('\n')
print(classification_report(y_o_test,yoPredNB))

Oversampled (x) classes
[[5867  181]
 [3668 2311]]


              precision    recall  f1-score   support

           0       0.62      0.97      0.75      6048
           1       0.93      0.39      0.55      5979

    accuracy                           0.68     12027
   macro avg       0.77      0.68      0.65     12027
weighted avg       0.77      0.68      0.65     12027



In [10]:
# running descision tree classifiers
from sklearn.tree import DecisionTreeClassifier

In [11]:
# for unbalanced class data
classifierdf = DecisionTreeClassifier()
classifierdf.fit(X_df_train, y_df_train)
ydfPredDT = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredDT))
print('\n')
print(classification_report(y_df_test,ydfPredDT))

Unbalanced classes
[[3274  101]
 [ 115 5872]]


              precision    recall  f1-score   support

           0       0.97      0.97      0.97      3375
           1       0.98      0.98      0.98      5987

    accuracy                           0.98      9362
   macro avg       0.97      0.98      0.98      9362
weighted avg       0.98      0.98      0.98      9362



In [16]:
# for undersampled class data
classifieru = DecisionTreeClassifier()
classifieru.fit(X_u_train, y_u_train)
yuPredDT = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredDT))
print('\n')
print(classification_report(y_u_test,yuPredDT))

NameError: name 'DecisionTreeClassifier' is not defined

In [13]:
# for oversampled class data
classifier = DecisionTreeClassifier()
classifier.fit(X_o_train, y_o_train)
yoPredDT = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredDT))
print('\n')
print(classification_report(y_o_test,yoPredDT))

Oversampled (x) classes
[[5984   64]
 [ 116 5863]]


              precision    recall  f1-score   support

           0       0.98      0.99      0.99      6048
           1       0.99      0.98      0.98      5979

    accuracy                           0.99     12027
   macro avg       0.99      0.99      0.99     12027
weighted avg       0.99      0.99      0.99     12027



In [14]:
# running KNN classifiers
from sklearn.neighbors import KNeighborsClassifier

In [15]:
# for unbalanced class data
classifierdf = KNeighborsClassifier(n_neighbors=1)
classifierdf.fit(X_df_train, y_df_train)
ydfPredKNN = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredKNN))
print('\n')
print(classification_report(y_df_test,ydfPredKNN))

Unbalanced classes
[[3295   80]
 [ 103 5884]]


              precision    recall  f1-score   support

           0       0.97      0.98      0.97      3375
           1       0.99      0.98      0.98      5987

    accuracy                           0.98      9362
   macro avg       0.98      0.98      0.98      9362
weighted avg       0.98      0.98      0.98      9362



In [16]:
# for undersampled class data
classifieru = KNeighborsClassifier(n_neighbors=1)
classifieru.fit(X_u_train, y_u_train)
yuPredKNN = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredKNN))
print('\n')
print(classification_report(y_u_test,yuPredKNN))

Undersampled classes
[[3278   85]
 [  87 3247]]


              precision    recall  f1-score   support

           0       0.97      0.97      0.97      3363
           1       0.97      0.97      0.97      3334

    accuracy                           0.97      6697
   macro avg       0.97      0.97      0.97      6697
weighted avg       0.97      0.97      0.97      6697



In [17]:
# for oversampled class data
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(X_o_train, y_o_train)
yoPredKNN = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredKNN))
print('\n')
print(classification_report(y_o_test,yoPredKNN))

Oversampled (x) classes
[[5989   59]
 [  85 5894]]


              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6048
           1       0.99      0.99      0.99      5979

    accuracy                           0.99     12027
   macro avg       0.99      0.99      0.99     12027
weighted avg       0.99      0.99      0.99     12027



In [9]:
# running random forest classifier
from sklearn.ensemble import RandomForestClassifier

In [10]:
# for unbalanced class data
classifierdf = RandomForestClassifier(n_estimators=100)
classifierdf.fit(X_df_train, y_df_train)
ydfPredRF = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredRF))
print('\n')
print(classification_report(y_df_test,ydfPredRF))

Unbalanced classes
[[15221   918]
 [ 1893  7218]]


              precision    recall  f1-score   support

           0       0.89      0.94      0.92     16139
           1       0.89      0.79      0.84      9111

    accuracy                           0.89     25250
   macro avg       0.89      0.87      0.88     25250
weighted avg       0.89      0.89      0.89     25250



In [11]:
# for undersampled class data
classifieru = RandomForestClassifier(n_estimators=100)
classifieru.fit(X_u_train, y_u_train)
yuPredRF = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredRF))
print('\n')
print(classification_report(y_u_test,yuPredRF))

Undersampled classes
[[8090 1019]
 [1091 8003]]


              precision    recall  f1-score   support

           0       0.88      0.89      0.88      9109
           1       0.89      0.88      0.88      9094

    accuracy                           0.88     18203
   macro avg       0.88      0.88      0.88     18203
weighted avg       0.88      0.88      0.88     18203



In [12]:
# for oversampled class data
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_o_train, y_o_train)
yoPredRF = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredRF))
print('\n')
print(classification_report(y_o_test,yoPredRF))

Oversampled (x) classes
[[14664  1470]
 [ 1152 15010]]


              precision    recall  f1-score   support

           0       0.93      0.91      0.92     16134
           1       0.91      0.93      0.92     16162

    accuracy                           0.92     32296
   macro avg       0.92      0.92      0.92     32296
weighted avg       0.92      0.92      0.92     32296



In [22]:
# running stochastic gradient gescent classifier
from sklearn.linear_model import SGDClassifier

In [23]:
# for unbalanced class data
classifierdf = SGDClassifier()
classifierdf.fit(X_df_train, y_df_train)
ydfPredSGDC = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredSGDC))
print('\n')
print(classification_report(y_df_test,ydfPredSGDC))

Unbalanced classes
[[3269  106]
 [  79 5908]]


              precision    recall  f1-score   support

           0       0.98      0.97      0.97      3375
           1       0.98      0.99      0.98      5987

    accuracy                           0.98      9362
   macro avg       0.98      0.98      0.98      9362
weighted avg       0.98      0.98      0.98      9362



In [24]:
# for undersampled class data
classifieru = SGDClassifier()
classifieru.fit(X_u_train, y_u_train)
yuPredSGDC = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredSGDC))
print('\n')
print(classification_report(y_u_test,yuPredSGDC))

Undersampled classes
[[3257  106]
 [  56 3278]]


              precision    recall  f1-score   support

           0       0.98      0.97      0.98      3363
           1       0.97      0.98      0.98      3334

    accuracy                           0.98      6697
   macro avg       0.98      0.98      0.98      6697
weighted avg       0.98      0.98      0.98      6697



In [25]:
# for oversampled class data
classifier = SGDClassifier()
classifier.fit(X_o_train, y_o_train)
yoPredSGDC = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredSGDC))
print('\n')
print(classification_report(y_o_test,yoPredSGDC))

Oversampled (x) classes
[[5916  132]
 [  77 5902]]


              precision    recall  f1-score   support

           0       0.99      0.98      0.98      6048
           1       0.98      0.99      0.98      5979

    accuracy                           0.98     12027
   macro avg       0.98      0.98      0.98     12027
weighted avg       0.98      0.98      0.98     12027



In [26]:
# running gradient boosting classifier
from sklearn.ensemble import GradientBoostingClassifier

In [27]:
# for unbalanced class data
classifierdf = GradientBoostingClassifier()
classifierdf.fit(X_df_train, y_df_train)
ydfPredGBC = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredGBC))
print('\n')
print(classification_report(y_df_test,ydfPredGBC))

Unbalanced classes
[[3182  193]
 [  19 5968]]


              precision    recall  f1-score   support

           0       0.99      0.94      0.97      3375
           1       0.97      1.00      0.98      5987

    accuracy                           0.98      9362
   macro avg       0.98      0.97      0.98      9362
weighted avg       0.98      0.98      0.98      9362



In [28]:
# for undersampled class data
classifieru = GradientBoostingClassifier()
classifieru.fit(X_u_train, y_u_train)
yuPredGBC = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredGBC))
print('\n')
print(classification_report(y_u_test,yuPredGBC))

Undersampled classes
[[3198  165]
 [  13 3321]]


              precision    recall  f1-score   support

           0       1.00      0.95      0.97      3363
           1       0.95      1.00      0.97      3334

    accuracy                           0.97      6697
   macro avg       0.97      0.97      0.97      6697
weighted avg       0.97      0.97      0.97      6697



In [29]:
# for oversampled class data
classifier = GradientBoostingClassifier()
classifier.fit(X_o_train, y_o_train)
yoPredGBC = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredGBC))
print('\n')
print(classification_report(y_o_test,yoPredGBC))

Oversampled (x) classes
[[5770  278]
 [  26 5953]]


              precision    recall  f1-score   support

           0       1.00      0.95      0.97      6048
           1       0.96      1.00      0.98      5979

    accuracy                           0.97     12027
   macro avg       0.98      0.97      0.97     12027
weighted avg       0.98      0.97      0.97     12027



In [28]:
# exporting data

In [29]:
yoPredRF

array(['0', '0', '0', ..., '0', '1', '1'], dtype=object)

In [30]:
y_o_test

74563    0
75060    0
84032    0
54059    0
42940    0
        ..
83404    0
11661    0
72326    0
45077    1
43015    1
Name: RECALL_CLASS, Length: 32296, dtype: object

In [31]:
dfResultsRFo =  pd.DataFrame(data={"ACTUAL":y_o_test, "PREDICTED":yoPredRF, "INDEX":y_o_test.index}, 
                             index=y_o_test.index)
dfResultsRFo.head()

,ACTUAL,PREDICTED,INDEX
74563,0,0,74563
75060,0,0,75060
84032,0,0,84032
54059,0,0,54059
42940,0,1,42940


In [32]:
odf["IND2"] = odf.index
dfResultsRFo = dfResultsRFo.merge(odf, left_on="INDEX", right_on="IND2")

In [33]:
dfResultsRFo.head()

,ACTUAL,PREDICTED,INDEX,MDR_REPORT_KEY,MDR_TEXT_KEY,TEXT_TYPE_CODE,PATIENT_SEQUENCE_NUMBER,DATE_REPORT,FOI_TEXT,textLen,...,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,year,STD_MANUFACTURER,STD_DEVICE,RECALL_CLASS,CENSORED,_merge,CLEAN_FOI_TEXT,IND2
0,0,0,74563,9316974,166222339,D,1,NaN,IT WAS REPORTED THAT THE PATIENT PRESENTED REM...,NaN,...,NaN,R,2019,ST. JUDE,StJUDE_DURATA,0,2019-11-13,both,it report the patient present remot via merlin...,74563
1,0,0,75060,9387537,168532817,D,1,NaN,IT WAS REPORTED THAT THE PATIENT PRESENTED FOR...,NaN,...,NaN,R,2019,ST. JUDE,StJUDE_QUARTET,0,2019-11-27,both,it report the patient present implant procedur...,75060
2,0,0,84032,10242160,197906174,D,1,NaN,IT WAS REPORTED THAT DURING THE IMPLANT PROCED...,NaN,...,DA,Y,2020,MEDTRONIC,MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI,0,2020-07-07,both,it report dure implant procedur coil the right...,84032
3,0,0,54059,6675364,78670967,D,1,NaN,IT WAS REPORTED THAT THE PATIENT PRESENTED VIA...,NaN,...,NaN,R,2017,ST. JUDE,StJUDE_DURATA,0,2017-06-28,both,it report the patient present via merlinnet tr...,54059
4,0,0,54059,5862097,51691721,D,1,NaN,IT WAS REPORTED THAT THE PATIENT EXPERIENCED S...,NaN,...,DA,*,2016,MEDTRONIC,MEDTRONIC_VIVA,1,2016-08-09,both,it report the patient experienc syncop ventric...,54059


In [35]:
dfResultsRFo.to_pickle("./dfResultsRFo")

In [13]:
# set 2

In [26]:
# for oversampled class data
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_o_train, y_o_train)

RandomForestClassifier()

In [27]:
yo2PredRF = classifier.predict(X_o2_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o2_test,yo2PredRF))
print('\n')
print(classification_report(y_o2_test,yo2PredRF))

Oversampled (x) classes
[[600 158]
 [391 391]]


              precision    recall  f1-score   support

           0       0.61      0.79      0.69       758
           1       0.71      0.50      0.59       782

    accuracy                           0.64      1540
   macro avg       0.66      0.65      0.64      1540
weighted avg       0.66      0.64      0.64      1540



In [36]:
dfResultsRFo2 =  pd.DataFrame(data={"ACTUAL":y_o2_test, "PREDICTED":yo2PredRF, "INDEX":y_o2_test.index}, 
                             index=y_o2_test.index)
dfResultsRFo2.head()

,ACTUAL,PREDICTED,INDEX
86641,1,1,86641
72227,1,1,72227
1618,1,1,1618
1591,0,0,1591
3378,0,0,3378


In [37]:
odf2["IND2"] = odf2.index
dfResultsRFo2 = dfResultsRFo2.merge(odf2, left_on="INDEX", right_on="IND2")

In [38]:
dfResultsRFo2

,ACTUAL,PREDICTED,INDEX,MDR_REPORT_KEY,MDR_TEXT_KEY,TEXT_TYPE_CODE,PATIENT_SEQUENCE_NUMBER,DATE_REPORT,FOI_TEXT,textLen,...,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,year,STD_MANUFACTURER,STD_DEVICE,RECALL_CLASS,CENSORED,_merge,CLEAN_FOI_TEXT,IND2
0,1,1,86641,10521026,206505776,D,1,NaN,IT WAS REPORTED THAT THE PATIENT DEVELOPED A P...,NaN,...,NaN,*,2020,MEDTRONIC,MEDTRONIC_VISIA,1,2020-09-11,both,it report the patient develop pocket infect ap...,86641
1,1,1,72227,8926164,156895660,D,1,NaN,THE PATIENT REPORTED THAT THE IMPLANTABLE CARD...,NaN,...,NaN,*,2019,MEDTRONIC,MEDTRONIC_VISIA,1,2019-08-23,both,the patient report the implant cardiovert defi...,72227
2,1,1,1618,630955,412089,D,1,NaN,RETURNED FOR BATTERY DEPLETION AND SUBSEQUENTL...,NaN,...,4 YR,Y,2004,MEDTRONIC,MEDTRONIC_MICRO_JEWEL,1,2004-01-09,both,return batteri deplet subsequ test of specif a...,1618
3,0,0,1591,2951858,18619649,D,1,NaN,IT WAS REPORTED THAT DURING THE IMPLANT PROCED...,NaN,...,DA,Y,2013,MEDTRONIC,MEDTRONIC_TRANSVENE,0,2013-02-09,both,it report dure implant procedur physician unab...,1591
4,0,0,3378,825999,587300,D,1,NaN,THE LEAD WAS RETURNED TO THE MANUFACTURER FOR ...,NaN,...,24 MO,Y,2007,MEDTRONIC,MEDTRONIC_SPRINT_FIDELIS,1,2007-03-09,both,the lead return the manufactur evalu analyz su...,3378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,0,1,3918,7092972,94068456,D,1,NaN,IT WAS REPORTED THAT THE PATIENT DEVELOPED FEV...,NaN,...,DA,*,2017,MEDTRONIC,MEDTRONIC_TRANSVENE,0,2017-12-07,both,it report the patient develop fever wa treat a...,3918
1971,0,0,2693,760307,540626,D,1,NaN,HIGH IMPEDANCE AND 'INADEQUATE THERAPIES' WERE...,NaN,...,12 MO,Y,2006,MEDTRONIC,MEDTRONIC_SPRINT_FIDELIS,1,2006-09-07,both,high imped inadequ therapi report the short in...,2693
1972,0,0,2693,331998,189470,D,1,NaN,"INAPPROPRIATE VF THERAPY, SENSING AND CAPTURE ...",NaN,...,4 YR,Y,2000,MEDTRONIC,MEDTRONIC_TRANSVENE,0,2000-11-13,both,inappropri vf therapi sens captur difficulti,2693
1973,0,0,3188,785137,556255,D,1,NaN,IT WAS REPORTED THAT THE RIGHT VENTRICULAR LEA...,NaN,...,10 YR,R,2006,MEDTRONIC,MEDTRONIC_TRANSVENE,0,2006-11-10,both,it report the right ventricular lead model exp...,3188


In [39]:
dfResultsRFo2.to_pickle("./dfResultsRFo2")